<a href="https://colab.research.google.com/github/kla55/Pytorch_learning/blob/main/Sentiment_analysis_with_BERT/Sentiment_analysis_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
pd.options.mode.chained_assignment = None


In [11]:
dset_smile_url='https://archive.org/download/misc-dataset/smile-annotations-final.csv'
df=pd.read_csv(dset_smile_url,header=None)
df.columns=['id','text','category']
# df.set_index('id', inplace=True)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3085 entries, 0 to 3084
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3085 non-null   int64 
 1   text      3085 non-null   object
 2   category  3085 non-null   object
dtypes: int64(1), object(2)
memory usage: 72.4+ KB


,id,text,category
0,611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
...,...,...,...
3080,613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy
3081,613294681225621504,@britishmuseum Upper arm guard?,nocode
3082,615246897670922240,@MrStuchbery @britishmuseum Mesmerising.,happy
3083,613016084371914753,@NationalGallery The 2nd GENOCIDE against #Bia...,not-relevant


In [12]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [14]:
df = df[~df.category.str.contains('\|')]
df = df[df.category != 'nocode']

In [15]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [16]:
possible_labels = df.category.unique()

In [27]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

label_encoder = preprocessing.LabelEncoder()
df['label']= label_encoder.fit_transform(df['category'])
df

,id,text,category,species,label
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,2,2
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,2,2
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy,2,2
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,2,2
5,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,2,2
...,...,...,...,...,...
3078,611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,3,3
3079,612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,2,2
3080,613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,2,2
3082,615246897670922240,@MrStuchbery @britishmuseum Mesmerising.,happy,2,2


In [28]:
print(df['label'].unique())
print(df['category'].unique())

[2 3 0 1 4 5]
['happy' 'not-relevant' 'angry' 'disgust' 'sad' 'surprise']


In [29]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.15,
    random_state = 17,
    stratify = df.label.values
)

In [37]:
df['data_type'] = 'not_set'
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [39]:
df.groupby(['category', 'label', 'data_type']).count()

id  text  species
category     label data_type                    
angry        0     train       48    48       48
                   val          9     9        9
disgust      1     train        5     5        5
                   val          1     1        1
happy        2     train      966   966      966
                   val        171   171      171
not-relevant 3     train      182   182      182
                   val         32    32       32
sad          4     train       27    27       27
                   val          5     5        5
surprise     5     train       30    30       30
                   val          5     5        5

In [40]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [41]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [42]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [45]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens = True,
    return_attention_mask = True,
    truncation=True,
    padding=True,
    max_length = 256,
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].text.values,
    add_special_tokens = True,
    return_attention_mask = True,
    truncation=True,
    padding=True,
    max_length = 256,
    return_tensors = 'pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)

In [47]:
encoded_data_train

{'input_ids': tensor([[  101, 16092,  3897,  ...,     0,     0,     0],
        [  101,  1030, 10682,  ...,     0,     0,     0],
        [  101,  1030,  2329,  ...,     0,     0,     0],
        ...,
        [  101,  1030,  1035,  ...,     0,     0,     0],
        [  101,  1523,  1030,  ...,     0,     0,     0],
        [  101,  1030,  2120,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}